1. Przygotowanie danych

Używając instrukcji zawartej w rozdziale 3.1 można wygenerować zdarzenia w programie GENIE. Wygenerowane dane zostaną zapisane w formacie .ROOT, który należy zamieniać na format RooTracker za pomocą komendy:
gntpc -i <nazwa pliku.root> -f rootracker

Używając tak przygotowanego pliku interesujące nas dane można przepisać do formatu .csv.

In [ ]:
import ROOT
import csv
import math
import re

In [ ]:
nu_file = ROOT.TFile("plik.gtrac_mu.root")

In [ ]:
#Lista kodów PDG cząstek neutralnych, które chcemy usunąć ze zbioru danych
neutral_particles = [
    -12, 12, -14, 14, -16, 16,
    -3122, 3122, -3212, 3212, -421, 421, -311, 311,
    -130, 130, 2000000101, 1000180400, 130, 310,
    3322, -3322, 221, 331, 443, 2112, -2112
]

In [ ]:
#Funkcja do wyciągnięcia ze zdarzenia typu (CC/NC) oraz rodzaju (RES,DIS,QES) oddziaływania
def parse_interaction_code(code_str):
    current_match = re.search(r'\[(NC|CC)\]', code_str)
    current = current_match.group(1) if current_match else "UNKNOWN"

    if "DIS" in code_str:
        interaction = "DIS"
    elif "RES" in code_str:
        interaction = "RES"
    elif "QES" in code_str:
        interaction = "QES"
    else:
        interaction = "OTHER"

    return current, interaction

In [ ]:
def extract_data(input_file, output_file, particles_list):
    tree = input_file.Get("gRooTracker") #Wgranie drzewa ROOT z danymi
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        header = ["EvtNum", "CurrentType", "InteractionType"]
        for i in range(4):
            header.append(f"P{i+1}_id")
            header.append(f"P{i+1}_px")
            header.append(f"P{i+1}_py")
            header.append(f"P{i+1}_pz")
            header.append(f"P{i+1}_E")
        writer.writerow(header)
        
        for entry in tree:
            
            interaction_str = tree.EvtCode.GetString().Data() if hasattr(tree, "EvtCode") else "proc:Unknown" #Czytanie inforamcji o oddziaływaniu w zdarzeniu
            current_type, interaction_type = parse_interaction_code(interaction_str) 

            row = [tree.EvtNum, current_type, interaction_type]
            particle_data = []
            #Lista zawierajaca czteropędy każdej cząstki
            four_vectors = tree.StdHepP4
            #Iteracja po cząstkach w oddziaływaniu
            for particle in range(tree.StdHepN):
                particle_type = tree.StdHepPdg[particle] #Kod PDG czastki
                if tree.StdHepStatus[particle] != 1:  #Sprawdzanie czy cząstka jest stanem finalnym
                    continue
            #Dla protonu sprawdzamy, czy jego pęd jest większy od 400 MeV, w przeciwnym przypadku nie zostaje zapisany
                if particle_type == 2212:
                    px = four_vectors[4 * particle + 0]
                    py = four_vectors[4 * particle + 1]
                    pz = four_vectors[4 * particle + 2]
                    momentum = math.sqrt(px**2 + py**2 + pz**2)
                    if momentum < 0.4:
                        continue

            #Sprawdzenie czy kod PDG cząstki znajduje się w liście cząstek neutralnych
                if particle_type in particles_list:
                    continue

            #Dodajemy kod cząstki oraz jej czteropęd do listy
                particle_data.append(particle_type)
                for counter in range(4):
                    particle_data.append(four_vectors[4 * particle + counter])

            
            if particle_data:
                writer.writerow(row + particle_data)


In [ ]:
extract_data(nu_file, 'test_data_tau.csv', neutral_particles)